In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [3]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-10
DEST_DIR: ..\data

source_path: ..\data\2025-04-10_df_OHLCV_clean.parquet
dest_path: ..\data\2025-04-10_df_perf_ratios.parquet


In [4]:
import pandas as pd

# Get tickers from df_finviz.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz.parquet')
tickers = df_finviz.index.to_list()

In [5]:
import pandas as pd

pd.set_option('display.width', 1000) 

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
display(df)

Open    High     Low   Close  Adj Close   Volume    Adj Open    Adj High     Adj Low
Symbol Date                                                                                              
FNV    2025-04-10  151.81  159.46  150.74  157.48     157.48  1689960  151.810000  159.460000  150.740000
       2025-04-09  148.04  152.50  145.39  150.22     150.22  1885000  148.040000  152.500000  145.390000
       2025-04-08  150.50  150.69  142.08  143.04     143.04  1237500  150.500000  150.690000  142.080000
       2025-04-07  141.48  150.19  140.03  145.45     145.45  1431800  141.480000  150.190000  140.030000
       2025-04-04  155.00  155.97  144.76  145.02     145.02  1962700  155.000000  155.970000  144.760000
...                   ...     ...     ...     ...        ...      ...         ...         ...         ...
PNFP   2024-04-17   80.08   80.33   78.32   78.33      77.61   214200   79.343914   79.591616   77.600092
       2024-04-16   79.00   79.68   78.18   78.95      78.23   289700   78.279544   78.953343   77.467022
       2024-04-15   80.40   81.57   78.85   79.79      79.06   450300   79.664419   80.823715   78.128600
       2024-04-12   79.39   80.06   78.89   79.95      79.22   390900   78.665113   79.328996   78.169679
       2024-04-11   80.09   80.64   78.98   80.46      79.72   309500   79.353403   79.898345   78.253612

[386038 rows x 9 columns]

In [6]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
FNV    2025-04-10     157.48
       2025-04-09     150.22
       2025-04-08     143.04
       2025-04-07     145.45
       2025-04-04     145.02
...                      ...
PNFP   2024-04-17      77.61
       2024-04-16      78.23
       2024-04-15      79.06
       2024-04-12      79.22
       2024-04-11      79.72

[386038 rows x 1 columns]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 386038 entries, ('FNV', Timestamp('2025-04-10 00:00:00')) to ('PNFP', Timestamp('2024-04-11 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  386038 non-null  float64
dtypes: float64(1)
memory usage: 4.5+ MB


In [18]:
df.loc['AEM']

,Adj Close
Date,
2025-04-10,111.55
2025-04-09,105.80
2025-04-08,100.13
2025-04-07,99.63
2025-04-04,99.48
...,...
2024-04-17,61.82
2024-04-16,61.08
2024-04-15,60.77


In [8]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-10     190.42
       2025-04-09     198.85
       2025-04-08     172.42
       2025-04-07     181.46
       2025-04-04     188.38
...                      ...
SKYY   2024-04-17      90.76
       2024-04-16      91.55
       2024-04-15      91.44
       2024-04-12      93.68
       2024-04-11      96.05

[386038 rows x 1 columns]


In [9]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [26]:
print(list_dfs[0])
print(list_dfs[0].loc['AEM'])

                   Adj Close
Symbol Date                 
A      2025-04-08      99.29
       2025-04-09     107.05
       2025-04-10      99.95
AA     2025-04-08      22.57
       2025-04-09      25.86
...                      ...
ZTO    2025-04-09      17.84
       2025-04-10      17.66
ZTS    2025-04-08     144.41
       2025-04-09     153.55
       2025-04-10     147.00

[4614 rows x 1 columns]
            Adj Close
Date                 
2025-04-08     100.13
2025-04-09     105.80
2025-04-10     111.55


In [10]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A      0.894499    1.943905  1.173177  -1.535120   -2.487122  0.772672   -5.437513    -6.590255   0.416167   -5.413622    -6.408665   0.386686   -4.439156    -5.311061   0.450321   -3.890923    -4.665131   0.488188    -2.040525     -2.642918    0.693381    -1.136186     -1.487523    0.815134
AA     3.152705    8.768051  1.781120  -1.015973   -1.874118  0.841268   -5.029465    -6.429914   0.425134   -5.839947    -7.069826   0.348189   -3.152110    -4.024909   0.581710   -2.805136    -3.522187   0.605925    -1.815445     -2.345565    0.720420    -0.564898     -0.781285    0.908581
AAL    2.449555    6.266638  1.558276   1.020816    2.048770  1.194852   -1.2829

In [29]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1538 entries, A to ZTS
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sharpe 3d     1538 non-null   float64
 1   Sortino 3d    1538 non-null   float64
 2   Omega 3d      1399 non-null   float64
 3   Sharpe 5d     1538 non-null   float64
 4   Sortino 5d    1538 non-null   float64
 5   Omega 5d      1535 non-null   float64
 6   Sharpe 10d    1538 non-null   float64
 7   Sortino 10d   1538 non-null   float64
 8   Omega 10d     1538 non-null   float64
 9   Sharpe 15d    1538 non-null   float64
 10  Sortino 15d   1538 non-null   float64
 11  Omega 15d     1538 non-null   float64
 12  Sharpe 30d    1538 non-null   float64
 13  Sortino 30d   1538 non-null   float64
 14  Omega 30d     1538 non-null   float64
 15  Sharpe 60d    1538 non-null   float64
 16  Sortino 60d   1538 non-null   float64
 17  Omega 60d     1538 non-null   float64
 18  Sharpe 120d   1538 non-null   floa

In [ ]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            # NaN is from Omega ratio where daily loss series are all zero
            # +Inf is from Sharpe ratio where daily negative excess return series are all zero
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            # -Inf is from Sharpe ratio where daily excess return series are the same, therefore standard deviation is zero
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sharpe 3d
  -Inf: ['BSCP', 'SHV']
Column: Sortino 3d
  Inf: ['ACI', 'ACT', 'ADC', 'ADM', 'AEE', 'AEM', 'AEP', 'AGI', 'AJG', 'AMT', 'AOS', 'ASND', 'ATO', 'AU', 'AZO', 'BAH', 'BILI', 'BJ', 'BRO', 'BTI', 'CEF', 'CHD', 'CHE', 'CI', 'CL', 'CLX', 'CMF', 'CMS', 'COKE', 'CPRT', 'CTRE', 'CWST', 'DUK', 'DVA', 'DXCM', 'EA', 'EBAY', 'EFAV', 'ELV', 'ENB', 'ERIE', 'ESLT', 'EVRG', 'FE', 'FICO', 'FMS', 'FNV', 'FYBR', 'GD', 'GDX', 'GDXJ', 'GFI', 'GLD', 'GLDM', 'GOLD', 'HII', 'HMY', 'HRB', 'HUM', 'HYD', 'IAU', 'IDA', 'JD', 'KDP', 'KGC', 'KMB', 'KNSL', 'KO', 'KVUE', 'LI', 'LMT', 'LRN', 'MCD', 'MCK', 'MO', 'MOH', 'MUSA', 'NEAR', 'NEM', 'NGG', 'NOC', 'NTES', 'O', 'OHI', 'OLLI', 'OPCH', 'ORLY', 'OTIS', 'PAAS', 'PG', 'PGR', 'PHYS', 'PPC', 'PPL', 'PSLV', 'PSN', 'PTLC', 'RELX', 'RGLD', 'RLI', 'ROL', 'ROST', 'RSG', 'RTO', 'RYAN', 'SAIC', 'SBAC', 'SBS', 'SCI', 'SFM', 'SGOL', 'S

In [12]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'ACI'

#### Use Cell below to check the inf, -inf and NaN replacement

In [13]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       15.376310
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d       -5.348699
Sortino 5d      -6.430298
Omega 5d         0.412540
Sharpe 10d      -0.871062
Sortino 10d     -1.118699
Omega 10d        0.872086
Sharpe 15d       0.608892
Sortino 15d      0.813630
Omega 15d        1.102484
Sharpe 30d       0.426259
Sortino 30d      0.621065
Omega 30d        1.069564
Sharpe 60d       0.774130
Sortino 60d      1.117853
Omega 60d        1.137099
Sharpe 120d      1.258667
Sortino 120d     1.928283
Omega 120d       1.229709
Sharpe 250d      0.262093
Sortino 250d     0.384200
Omega 250d       1.045210
Name: ACI, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d          15.376310
Sortino 3d      63320.772323
Omega 3d         5642.062750
Sharpe 5d          -5.348699
Sortino 5d         -6.430298
Omega 5d            0.412540
Sharpe 10d         -0.871062
Sortino 10d        -1.118699
Omega 10d           0.872086
Sharpe 15d          0.608892
Sortino 15d         0.813630
Omega 15d           1.102484
Sharpe 30d          0.426259
Sortino 30d         0.621065
Omega 30d           1.069564
Sharpe 60d          0.774130
Sortino 60d         1.117853
Omega 60d           1.137099
Sharpe 120d         1.258667
Sortino 120d        1.928283
Omega 120d          1.229709
Sharpe 250d         0.262093
Sortino 250d        0.384200
Omega 250d          1.045210
Name: ACI, dtype: float64

In [14]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-10_df_perf_ratios.parquet


In [15]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['USFR','AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====